# Regridding ClimateDT to 

imports

In [1]:
import os

del os.environ["PROJ_DATA"]
del os.environ["ESMFMKFILE"]
del os.environ["_CONDA_SET_ESMFMKFILE"]

In [2]:
import earthkit.data
import earthkit.regrid
from earthkit.plots.interactive import Chart
from polytope.api import Client
import xesmf

import xdggs
import dask.array as da

import numpy as np
import healpix_geo.nested
import xarray as xr

## Download data

generate token (run once)

request data and load as `DataTree`

In [3]:
# Set True if you want to make a live request for the data, or false if you want to use the cached grib file
LIVE_REQUEST = False

For a description of the parameters, see [here](https://confluence.ecmwf.int/display/DDCZ/DestinE%2BClimateDT%2BParameters)

In [4]:
pressure_levels = [1000, 925, 850, 700, 600, 500, 400, 300, 250, 200, 150, 100, 70, 50, 30, 20, 10, 5, 1]
request = {
    "activity": "scenariomip",
    "class": "d1",
    "dataset": "climate-dt",
    "experiment": "ssp3-7.0",
    "generation": "1",
    "levtype": "pl",
    "date": "20210101",
    "model": "ifs-nemo",
    "expver": "0001",
    "param": "130",
    "realization": "1",
    "resolution": "high",
    "stream": "clte",
    "type": "fc",
    "time": "/".join(f"{n * 6:02d}00" for n in range(4)),
    "levelist": "/".join(str(_) for _ in pressure_levels),
}
request

{'activity': 'scenariomip',
 'class': 'd1',
 'dataset': 'climate-dt',
 'experiment': 'ssp3-7.0',
 'generation': '1',
 'levtype': 'pl',
 'date': '20210101',
 'model': 'ifs-nemo',
 'expver': '0001',
 'param': '130',
 'realization': '1',
 'resolution': 'high',
 'stream': 'clte',
 'type': 'fc',
 'time': '0000/0600/1200/1800',
 'levelist': '1000/925/850/700/600/500/400/300/250/200/150/100/70/50/30/20/10/5/1'}

In [5]:
data_file = "climate-dt-earthkit.grib"
if LIVE_REQUEST:
    data = earthkit.data.from_source("polytope", "destination-earth", request, address="polytope.lumi.apps.dte.destination-earth.eu", stream=False)
    data.to_target("file", data_file)
else:
    data = earthkit.data.from_source("file", data_file)

In [6]:
ds = data.to_xarray(chunks={"values": 4**8, "forecast_reference_time": -1, "level": -1})
ds

<xarray.Dataset> Size: 8GB
Dimensions:                  (forecast_reference_time: 4, level: 19,
                              values: 12582912)
Coordinates:
  * forecast_reference_time  (forecast_reference_time) datetime64[ns] 32B 202...
  * level                    (level) int64 152B 1 5 10 20 ... 700 850 925 1000
    latitude                 (values) float64 101MB dask.array<chunksize=(65536,), meta=np.ndarray>
    longitude                (values) float64 101MB dask.array<chunksize=(65536,), meta=np.ndarray>
Dimensions without coordinates: values
Data variables:
    t                        (forecast_reference_time, level, values) float64 8GB dask.array<chunksize=(4, 19, 65536), meta=np.ndarray>
Attributes:
    param:        t
    paramId:      130
    class:        d1
    stream:       clte
    levtype:      pl
    type:         fc
    expver:       0001
    date:         20210101
    time:         0
    domain:       g
    Conventions:  CF-1.8
    institution:  ECMWF

## decode

In [7]:
decoded = (
    ds.assign_coords(cell_ids=("values", da.arange(ds.sizes["values"], dtype="u8")))
    .dggs.decode({"grid_name": "healpix", "level": 10, "indexing_scheme": "nested"}, index_kind="moc")
)
decoded

<xarray.Dataset> Size: 8GB
Dimensions:                  (forecast_reference_time: 4, level: 19,
                              values: 12582912)
Coordinates:
  * forecast_reference_time  (forecast_reference_time) datetime64[ns] 32B 202...
  * level                    (level) int64 152B 1 5 10 20 ... 700 850 925 1000
    latitude                 (values) float64 101MB dask.array<chunksize=(65536,), meta=np.ndarray>
    longitude                (values) float64 101MB dask.array<chunksize=(65536,), meta=np.ndarray>
  * cell_ids                 (values) uint64 101MB dask.array<chunksize=(12582912,), meta=np.ndarray>
Dimensions without coordinates: values
Data variables:
    t                        (forecast_reference_time, level, values) float64 8GB dask.array<chunksize=(4, 19, 65536), meta=np.ndarray>
Indexes:
    cell_ids  HealpixIndex(level=10, indexing_scheme=nested, kind=moc)
Attributes:
    param:        t
    paramId:      130
    class:        d1
    stream:       clte
    levtype:      pl
    type:         fc
    expver:       0001
    date:         20210101
    time:         0
    domain:       g
    Conventions:  CF-1.8
    institution:  ECMWF

## regridding

In [8]:
def spherical_to_ellipsoidal(ds):
    source_grid = ds[["latitude", "longitude"]].load()
    grid_info = ds.dggs.grid_info

    cell_ids = np.arange(12 * 4**grid_info.level, dtype="uint64")
    longitude, latitude = healpix_geo.nested.healpix_to_lonlat(
        cell_ids, depth=grid_info.level, ellipsoid="WGS84"
    )
    target_grid = xr.Dataset(
        coords={
            "cell_ids": ("cells", cell_ids),
            "longitude": ("cells", longitude),
            "latitude": ("cells", latitude),
        }
    )

    regridder = xesmf.Regridder(
        source_grid,
        target_grid,
        method="nearest_s2d",
        locstream_in=True,
        locstream_out=True,
        periodic=True,
    )

    return regridder.regrid_dataset(ds, keep_attrs=True).dggs.decode(grid_info)

In [9]:
%%time
regridded = decoded.pipe(spherical_to_ellipsoidal)
regridded

/home/jovyan/grid4earth/.pixi/envs/default/lib/python3.13/site-packages/xarray/computation/apply_ufunc.py:450: PerformanceWarning: Regridding is increasing the number of chunks by a factor of 192.0, you might want to specify sizes in `output_chunks` in the regridder call. Default behaviour is to preserve the chunk sizes from the input (1, 65536).
  result_vars[name] = func(*variable_args)


CPU times: user 1min 23s, sys: 8.14 s, total: 1min 31s
Wall time: 1min 33s


<xarray.Dataset> Size: 8GB
Dimensions:                  (forecast_reference_time: 4, level: 19,
                              cells: 12582912)
Coordinates:
  * forecast_reference_time  (forecast_reference_time) datetime64[ns] 32B 202...
  * level                    (level) int64 152B 1 5 10 20 ... 700 850 925 1000
  * cell_ids                 (cells) uint64 101MB 0 1 2 ... 12582910 12582911
    longitude                (cells) float64 101MB 45.0 45.04 ... 315.0 315.0
    latitude                 (cells) float64 101MB 0.03747 0.07494 ... -0.03747
Dimensions without coordinates: cells
Data variables:
    t                        (forecast_reference_time, level, cells) float64 8GB dask.array<chunksize=(4, 19, 65536), meta=np.ndarray>
Indexes:
    cell_ids  HealpixIndex(level=10, indexing_scheme=nested, kind=pandas)
Attributes: (12/13)
    param:          t
    paramId:        130
    class:          d1
    stream:         clte
    levtype:        pl
    type:           fc
    ...             ...
    date:           20210101
    time:           0
    domain:         g
    Conventions:    CF-1.8
    institution:    ECMWF
    regrid_method:  nearest_s2d

## metadata convention

In [10]:
def as_cf(ds):
    grid_info = ds.dggs.grid_info
    metadata = {
        "grid_mapping_name": "healpix",
        "refinement_level": grid_info.level,
        "indexing_scheme": grid_info.indexing_scheme,
        "reference_body": "WGS84",
        "refinement_ratio": 4,
    }

    new = ds.assign_coords(crs=xr.Variable((), np.int8(0), metadata))
    new["cell_ids"].attrs = {"standard_name": "healpix", "units": "1"}

    var_attrs = {"coordinates": ds.dggs._name, "grid_mapping": "crs"}

    for name, var in new.variables.items():
        if name in {"cell_ids"}:
            continue

        var.attrs |= var_attrs

    return new

In [11]:
cf = as_cf(regridded)
cf["t"].attrs.pop("_earthkit", None)
cf

<xarray.Dataset> Size: 8GB
Dimensions:                  (forecast_reference_time: 4, level: 19,
                              cells: 12582912)
Coordinates:
  * forecast_reference_time  (forecast_reference_time) datetime64[ns] 32B 202...
  * level                    (level) int64 152B 1 5 10 20 ... 700 850 925 1000
  * cell_ids                 (cells) uint64 101MB 0 1 2 ... 12582910 12582911
    longitude                (cells) float64 101MB 45.0 45.04 ... 315.0 315.0
    latitude                 (cells) float64 101MB 0.03747 0.07494 ... -0.03747
    crs                      int8 1B 0
Dimensions without coordinates: cells
Data variables:
    t                        (forecast_reference_time, level, cells) float64 8GB dask.array<chunksize=(4, 19, 65536), meta=np.ndarray>
Indexes:
    cell_ids  HealpixIndex(level=10, indexing_scheme=nested, kind=pandas)
Attributes: (12/13)
    param:          t
    paramId:        130
    class:          d1
    stream:         clte
    levtype:        pl
    type:           fc
    ...             ...
    date:           20210101
    time:           0
    domain:         g
    Conventions:    CF-1.8
    institution:    ECMWF
    regrid_method:  nearest_s2d

## store to disk

In [ ]:
zarr_url = f"{data_file.removesuffix('.grib')}.zarr"
cf.to_zarr(zarr_url, mode="w")

/home/jovyan/grid4earth/.pixi/envs/default/lib/python3.13/site-packages/zarr/api/asynchronous.py:244: ZarrUserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


## reopen

In [ ]:
reloaded = xr.open_datatree(zarr_url, engine="zarr", chunks={})
reloaded